## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-06-12-51-00 +0000,nypost,Nonprofit bosses turned historic Park Avenue A...,https://nypost.com/2025/09/06/us-news/nonprofi...
1,2025-09-06-12-40-00 +0000,nypost,"Number of first responders, others with 9/11-l...",https://nypost.com/2025/09/06/us-news/number-o...
2,2025-09-06-12-30-00 +0000,nypost,9/11 Museum boss cashed big paychecks while he...,https://nypost.com/2025/09/06/us-news/ceo-of-e...
3,2025-09-06-12-25-56 +0000,nyt,Trump Administration Drops Biden Plan for Flig...,https://www.nytimes.com/2025/09/05/business/fl...
4,2025-09-06-12-21-48 +0000,nyt,Duchess of Kent Dies at 92; a Royal Who Comfor...,https://www.nytimes.com/2025/09/05/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2311/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
30,trump,49
212,new,16
199,will,13
182,immigration,10
599,war,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
135,2025-09-05-22-57-00 +0000,wsj,President Trump signed an executive order to r...,https://www.wsj.com/politics/national-security...,94
100,2025-09-06-01-39-40 +0000,nypost,Kilmar Abrego Garcia faces deportation to tiny...,https://nypost.com/2025/09/05/us-news/kilmar-a...,82
269,2025-09-05-15-43-16 +0000,nyt,Billionaire Crypto Investor Hits Out at Trump ...,https://www.nytimes.com/2025/09/05/business/tr...,80
143,2025-09-05-22-25-26 +0000,nyt,A Second Weak Jobs Report Challenges Trump’s C...,https://www.nytimes.com/2025/09/05/us/politics...,79
56,2025-09-06-09-00-00 +0000,wsj,Rep. Marjorie Taylor Greene is walking a tight...,https://www.wsj.com/politics/policy/marjorie-t...,79


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
135,94,2025-09-05-22-57-00 +0000,wsj,President Trump signed an executive order to r...,https://www.wsj.com/politics/national-security...
194,60,2025-09-05-20-08-47 +0000,nyt,Elon Musk Could Become First Trillionaire Unde...,https://www.nytimes.com/2025/09/05/business/el...
103,50,2025-09-06-01-20-10 +0000,nyt,South Koreans Are Swept Up in Immigration Raid...,https://www.nytimes.com/2025/09/05/us/georgia-...
238,44,2025-09-05-17-33-52 +0000,latimes,CBS' 'Face the Nation' will no longer edit tap...,https://www.latimes.com/entertainment-arts/bus...
127,42,2025-09-05-23-19-00 +0000,wsj,The IT unemployment rate fell to 4.5% in Augus...,https://www.wsj.com/articles/it-unemployment-f...
178,41,2025-09-05-20-58-03 +0000,nypost,Top secret SEAL Team 6 killed North Korean civ...,https://nypost.com/2025/09/05/us-news/top-secr...
94,37,2025-09-06-02-17-09 +0000,nypost,Ex-FBI official tipped off Chinese firm doing ...,https://nypost.com/2025/09/05/us-news/ex-fbi-o...
8,33,2025-09-06-12-06-46 +0000,nyt,Israel Targets More Buildings in Gaza City and...,https://www.nytimes.com/2025/09/06/world/middl...
270,32,2025-09-05-15-41-00 +0000,wsj,The Florida immigration detention facility kno...,https://www.wsj.com/politics/policy/florida-al...
72,32,2025-09-06-05-49-11 +0000,nypost,University of Kentucky cheerleader Laken Snell...,https://nypost.com/2025/09/06/us-news/universi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
